In [13]:
from kafka import KafkaProducer
import json
from yahoo_fin.stock_info import get_data
from time import sleep

import os
from dotenv import load_dotenv
load_dotenv()

True

# Kafka Producer creation

In [14]:
producer = KafkaProducer(bootstrap_servers=['18.119.159.115:9092'], value_serializer=lambda x: json.dumps(x).encode('utf-8'))

# AAPL's market data that's going to be send it to the consumer

In [15]:
df = get_data('AAPL', start_date='01/01/2019', end_date='01/01/2020', index_as_date = True, interval="1wk")
df.head()

,open,high,low,close,adjclose,volume,ticker
2019-01-01,38.722500,39.712502,35.500000,36.982498,35.542973,966947200,AAPL
2019-01-08,37.389999,38.632500,37.130001,37.500000,36.040325,725470000,AAPL
2019-01-15,37.567501,39.470001,37.512501,39.205002,37.678963,491411200,AAPL
2019-01-22,39.102501,39.532501,37.924999,39.075001,37.554020,554774800,AAPL
2019-01-29,39.062500,42.915001,38.527500,42.812500,41.146030,830400800,AAPL


In [16]:
df.sample(1).to_dict(orient='records')

[{'open': 72.48249816894531,
  'high': 73.41999816894531,
  'low': 72.37999725341797,
  'close': 73.4124984741211,
  'adjclose': 71.61502838134766,
  'volume': 100805600,
  'ticker': 'AAPL'}]

# Sends the data to the consumer

In [18]:
producer.flush()
x = 0
while True:
    dict_stock = df.sample(1).to_dict(orient='records')[0]
    producer.send(os.environ.get('KAFKA_TOPIC_NAME'), value=dict_stock)
    sleep(1)
    x += 1  
    if x == 5:
        break